In [1]:
# activate project environment
# include these lines of code in any future scripts/notebooks
#---
import Pkg
if !haskey(Pkg.installed(), "AA228FinalProject")
    jenv = joinpath(dirname(@__FILE__()), ".") # this assumes the notebook is in the same dir
    # as the Project.toml file, which should be in top level dir of the project. 
    # Change accordingly if this is not the case.
    Pkg.activate(jenv)
end
Pkg.instantiate()
Pkg.build("Cairo")
#---

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %  Updating registry at `~/.julia/registries/JuliaPOMDP`
  Updating git-repo `https://github.com/JuliaPOMDP/Registry`
  Building LibCURL ─→ `~/.julia/packages/LibCURL/OoXMv/deps/build.log`
  Building WinRPM ──→ `~/.julia/packages/WinRPM/Y9QdZ/deps/build.log`
  Building Homebrew → `~/.julia/packages/Homebrew/l8kUw/deps/build.log`
  Building Cairo ───→ `~/.julia/packages/Cairo/CXPG1/deps/build.log`


In [14]:
# import necessary packages
using AA228FinalProject
using TabularTDLearning
using POMDPs
using FIB # For the solver
using QMDP
using POMDPModels
using POMDPPolicies
using BeliefUpdaters
using ParticleFilters
using POMDPSimulators
using Cairo
using Gtk
using Random
using Printf

┌ Info: Precompiling POMDPModels [355abbd5-f08e-5560-ac9e-8b5f2592a0ca]
└ @ Base loading.jl:1186
┌ Warning: Module JSON with build ID 103108271525773 is missing from the cache.
│ This may mean JSON [682c06a0-de6a-54ab-a142-c8b1cf79cde6] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:941
┌ Warning: Module JSON with build ID 103108271525773 is missing from the cache.
│ This may mean JSON [682c06a0-de6a-54ab-a142-c8b1cf79cde6] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:941
┌ Info: Precompiling Compose [a81c6b42-2e10-5240-aca2-a61377ecd94b]
└ @ Base loading.jl:1186
┌ Warning: Module JSON with build ID 103108271525773 is missing from the cache.
│ This may mean JSON [682c06a0-de6a-54ab-a142-c8b1cf79cde6] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:941
┌ Info: Precompiling ColorSchemes [35d6a980-a343-548e-a6ea-1d62b119f2f4]
└ @ Base loading.jl:1186
┌ Warni

┌ Info: Recompiling stale cache file /Users/heathershen/.julia/compiled/v1.0/IntervalSets/2FHuv.ji for IntervalSets [8197267c-284f-5f27-9208-e0e47529a953]
└ @ Base loading.jl:1184
┌ Warning: Module Compat with build ID 103109992472183 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:941
┌ Info: Recompiling stale cache file /Users/heathershen/.julia/compiled/v1.0/ImageMetadata/dSTEq.ji for ImageMetadata [bc367c6b-8a6b-528e-b4bd-a4b897500b49]
└ @ Base loading.jl:1184
┌ Warning: Module ImageAxes with build ID 108874280816915 is missing from the cache.
│ This may mean ImageAxes [2803e5a7-5153-5ecf-9a86-9b4c37f5f5ac] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:941
┌ Info: Recompiling stale cache file /Users/heathershen/.julia/compiled/v1.0/ImageFiltering/MN5uU.ji for ImageFiltering [6a3955dd-da59-5b1f-98d4-e7296123d

In [4]:
sensor = Bumper()
config = 3 # 1,2, or 3
ds = DiscreteRoombaStateSpace(20, 20, 20)
v_steps = range(0.0, stop = 10.0, length = 10) |> collect
om_steps = range(-pi+0.01, stop = pi, length = 100) |> collect

m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(sspace=ds, aspace=[RoombaAct(v,om) for v in v_steps for om in om_steps], config=config))


RoombaPOMDP{Bumper,Bool}(Bumper(), RoombaMDP{DiscreteRoombaStateSpace,Array{RoombaAct,1}}
  v_max: Float64 10.0
  om_max: Float64 1.0
  dt: Float64 0.5
  contact_pen: Float64 -1.0
  time_pen: Float64 -0.1
  goal_reward: Float64 10.0
  stairs_penalty: Float64 -10.0
  config: Int64 3
  room: AA228FinalProject.Room
  sspace: DiscreteRoombaStateSpace
  aspace: Array{RoombaAct}((100,))
  _amap: Dict{RoombaAct,Int64}
)

In [15]:
num_particles = 2000
resampler = BumperResampler(num_particles)

spf = SimpleParticleFilter(m, resampler)

v_noise_coefficient = 2.0
om_noise_coefficient = 0.5

belief_updater = RoombaParticleFilter(spf, v_noise_coefficient, om_noise_coefficient);

solver = QMDPSolver(max_iterations=20, tolerance=1e-3)
policy = solve(solver, TigerPOMDP())

AlphaVectorPolicy{TigerPOMDP,Int64}(TigerPOMDP(-1.0, -100.0, 10.0, 0.85, 0.95), Array{Float64,1}[[135.803, 137.586], [147.694, 39.4478], [37.6943, 149.448]], [0, 1, 2])

In [11]:
# first seed the environment
Random.seed!(5)

# run the simulation
c = @GtkCanvas()
win = GtkWindow(c, "Roomba Environment", 600, 600)
for (t, step) in enumerate(stepthrough(m, policy, belief_updater, max_steps=100))
    @guarded draw(c) do widget
        
        # the following lines render the room, the particles, and the roomba
        ctx = getgc(c)
        set_source_rgb(ctx,1,1,1)
        paint(ctx)
        render(ctx, m, step)
        
        # render some information that can help with debugging
        # here, we render the time-step, the state, and the observation
        move_to(ctx,300,400)
        show_text(ctx, @sprintf("t=%d, state=%s, o=%.3f",t,string(step.s),step.o))
    end
    show(c)
    sleep(0.1) # to slow down the simulation
end

MethodError: MethodError: no method matching stateindex(::RoombaState)
Closest candidates are:
  stateindex(!Matched::POMDPModelTools.FullyObservablePOMDP{S,A}, !Matched::S) where {S, A} at /Users/heathershen/.julia/packages/POMDPModelTools/eHEjm/src/fully_observable_pomdp.jl:46
  stateindex(!Matched::POMDPModelTools.UnderlyingMDP{P,Int64,A}, !Matched::Int64) where {P, A} at /Users/heathershen/.julia/packages/POMDPModelTools/eHEjm/src/underlying_mdp.jl:29
  stateindex(!Matched::POMDPModelTools.UnderlyingMDP{P,Bool,A}, !Matched::Bool) where {P, A} at /Users/heathershen/.julia/packages/POMDPModelTools/eHEjm/src/underlying_mdp.jl:30
  ...